In [4]:
import ipyleaflet
from ipyleaflet import Map

In [1]:
from vleaflet import leaflet
%load_ext vuepy

In [2]:
%%vuepy_run --plugins leaflet
<template>
  <Slider v-model=mark_opacity.value description='opacity' :min=0 :max=1></Slider>
  <p>zoom {{ zoom.value }}</p>
  <p>marker on {{ mark_loc.value }}</p>
  <vl-map
    :center="[39.9042, 116.4074]"
    v-model:zoom="zoom.value"
    :max-zoom="13"
    :min-zoom="3"
    :zoom-control='False'
    :scroll-wheel-zoom='True'
    :keyboard="True"
    :basemap="basemaps.Esri.WorldTopoMap"
    ref=m
  >
    <template #controls>
      <vl-zoom-control position='topleft'></vl-zoom-control>
      <vl-scale-control position='topleft'></vl-scale-control>
      <vl-layers-control position='topleft' :collapsed='True'></vl-layers-control>
      <vl-fullscreen-control position='topleft'></vl-fullscreen-control>
      <vl-measure-control position='topleft'></vl-measure-control>
      <vl-widget-control position='topright'>
        <HBox>
          <Dropdown v-model=mapname.value :options=maps @change=change_map></Dropdown>
          <Button @click=clear_layers()>Clear Layers</Button>
          <Button @click=add_marker() icon='map-marker'>+</Button>
        </HBox>
      </vl-widget-control>
      <vl-search-control 
        url="https://nominatim.openstreetmap.org/search?format=json&q={s}" 
        position="topleft" @location_found=on_loc @feature_found=on_feat
        placeholder='search.'
      >
        <vl-marker v-model="mark_loc.value" name="mark1" @move="on_move" :opacity="mark_opacity.value" title="drag" >
          <template #popup>
            loc: {{ loc_ret.value }}
            feat: {{ feat_ret.value }}
          </template>
        </vl-marker>
      </vl-search-control>
      <vl-geoman-draw-control position='topright' :polyline="polyline" ></vl-geoman-draw-control>
    </template>
    <template #layers>
      <vl-marker v-model="mark_loc.value" name="mark1" @move="on_move" :opacity="mark_opacity.value" title="drag" >
        <template #popup>
          <div>Some html with a list of items
          <ul class='list-group'>
            <li class='list-group-item'>Item A</li>
            <li class='list-group-item'>Item B</li>
          </ul></div>
        </template>
      </vl-marker>
    </template>
  </vl-map>
</template>
<script lang='py'>
from ipyleaflet import basemaps, Marker
from vuepy import ref

p = (39.9042, 116.4074)
mark_opacity = ref(1.0)
mark_loc = ref(p)
m = ref(None)
maps = list(basemaps.Esri)
mapname = ref(None)

zoom = ref(13)

loc_ret = ref(None)
feat_ret = ref(None)

def on_loc(**kwargs):
    print('on_loc', kwargs) # {'event': 'locationfound', 'text': '杭州市, 浙江省, 中国', 'feature': None, 'location': [30.2489634, 120.2052342]}
    loc_ret.value = str(kwargs)

def on_feat(**kwargs):
    print('on_feat', kwargs) # {'event': 'locationfound', 'text': '杭州市, 浙江省, 中国', 'feature': None, 'location': [30.2489634, 120.2052342]}
    feat_ret.value = str(kwargs)
    

polyline = {
    "pathOptions": {
        "color": "red",
        "weight": 8,
        "opacity": 1.0
    }
}

def add_marker():
    m.value.add(Marker(location=p))

def change_map(e):
    esri = basemaps.Esri[e['new']]
    print(esri)
    print('---')
    m.value.basemap = esri

def clear_layers():
    m.value.clear_layers()

def on_move(**kwargs):
    print(kwargs)
    print("--")
</script>

Document(children=(Dom(children=(MessageWidget(message_options={'message': '', 'type': 'info', 'show_close': F…

In [21]:
from ipyleaflet import *
from ipywidgets import VBox, HTML

p = [30.600093873550072, 104.1064453125]
icon = AwesomeIcon(
    name='bus',
    marker_color='blue',
    icon_color='black'
)
mark = Marker(location=p, icon=icon, name=')
def at_move(*args, **kwargs):
    print(args)
    print(kwargs)
# mark.on_mouseup(at_move)
html_widget = HTML(
    value="""
        <div>Some html with a list of items
        <ul class='list-group'>
            <li class='list-group-item'>Item A</li>
            <li class='list-group-item'>Item B</li>
            <li class='list-group-item'>Item C</li>
        </ul></div>""",
    placeholder="",
    description="",
)
# mark.on_move(at_move)
mark.observe(lambda x: print(x), names='location')
mark.popup = html_widget

# popup
popup = Popup(
    location=p,
    child=HTML("hello"),
    auto_close=False,
    close_button=True,
)

zoom = ZoomControl(position='topright')

streetmap = basemap_to_tiles(basemaps.Esri.WorldStreetMap)
streetmap.name = 'world street'
layers = [streetmap, mark]
m = Map(center=p, zoom=10, scroll_wheel_zoom=True, layers=layers)  # 北京坐标
# m = Map(center=p, zoom=10, scroll_wheel_zoom=True, basemap=basemaps.Esri.WorldStreetMap)  # 北京坐标
# m = Map(center=p, zoom=10, zoom_control=False)
# m.add(mark)
m.add(zoom)
# m.add(popup)

control = LayersControl(position='topright', collapsed=False)
m.add(control)

measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)
m.add(measure)

draw_control = GeomanDrawControl()
draw_control.polyline =  {
    "pathOptions": {
        "color": "#6bc2e5",
        "weight": 8,
        "opacity": 1.0
    }
}
m.add(draw_control)

marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='darkgreen'))

sc = SearchControl(
  position="topright",
  url='https://nominatim.openstreetmap.org/search?format=json&q={s}',
  zoom=5,
  marker=marker
)
m.add(sc)

VBox(children=[m])

In [5]:
from ipyleaflet import basemaps, basemap_to_tiles

l = basemaps.Esri